# Understanding Elephant Cat Experiment

### Setup Environment

In [1]:
%matplotlib notebook
%config InlineBackend.figure_format = 'retina'

# native
import os
from os import listdir
from collections import defaultdict
from PIL import Image
import pprint as pp
import functools
import pickle
import re

# math
import numpy as np
from sklearn.metrics import accuracy_score

# plotting
import matplotlib
from matplotlib import pyplot as plt

# extra
from tqdm import tqdm

# pytorch
import torch
from torch.utils.data import Dataset, DataLoader
from torch.utils import model_zoo

import torchvision
import torchvision.transforms as transforms
import torchvision.datasets as datasets
import torchvision.models as models


#### Check Environment

In [2]:
requirements = {
    torch: '1',
    matplotlib: '3'
}

def check_requirements(requirements):
    for requirement in requirements:
        error_message = '{} environment does not match requirement'.format(requirement.__name__)
        assert (requirement.__version__[0] == requirements[requirement]), error_message

check_requirements(requirements)

#### Check CUDA

In [3]:
cuda = torch.cuda.is_available()

if cuda:
    torch.backends.cudnn.benchmark = True
    torch.set_default_tensor_type('torch.cuda.FloatTensor')

device = 'cuda' if cuda else 'cpu'

device

'cuda'

### Useful Functions

In [ ]:
class PlotGrid:
    def __init__(self, figsize=None):
        self.fig = plt.figure(figsize=figsize)
        self.ax = {}
        self.xlim = {}
        self.ylim = {}
        self.filled = {}
        self.grid = {}
    
    def plot(self, position_id, data, title=None, xlim=None, ylim=None, filled=None, grid=None):
        if position_id in self.ax:
            ax = self.ax[position_id]
        else:
            ax = self.fig.add_subplot(*position_id)

        # cache current values
        if title is None:
            title = ax.get_title()

        if xlim is not None:
            self.xlim[position_id] = xlim

        if ylim is not None:
            self.ylim[position_id] = ylim

        if filled is not None:
            self.filled[position_id] = filled
        
        if position_id not in self.filled:
            self.filled[position_id] = True

        if grid is not None:
            self.grid[position_id] = grid
        
        if position_id not in self.grid:
            self.grid[position_id] = True

        ax.cla()
        ax.clear()
        if type(data).__name__ == 'Image':
            ax.imshow(data)
        else:
            if hasattr(data, 'is_cuda') and data.is_cuda:
                data = data.cpu()
            if hasattr(data, 'numpy'):
                data = data.numpy()
            ax.plot(data)

            if self.filled[position_id]:
                ax.fill_between(range(len(data)), data)

            if self.grid[position_id]:
                ax.grid(True)

            # set xlim
            if position_id in self.xlim:
                ax.set_xlim(*self.xlim[position_id])

            # set ylim
            if position_id in self.ylim:
                ax.set_ylim(*self.ylim[position_id])
        
        # set title
        if title is not None:
            ax.set_title(title)

        self.fig.tight_layout()
        self.fig.canvas.draw()
        self.ax[position_id] = ax
    
    def prediction_plot(self, position_id, data, title=None, grid=None):
        if position_id in self.ax:
            ax = self.ax[position_id]
        else:
            ax = self.fig.add_subplot(*position_id)

        # cache current values
        if title is None:
            title = ax.get_title()

        if grid is not None:
            self.grid[position_id] = grid
        
        if position_id not in self.grid:
            self.grid[position_id] = True

        ax.cla()
        ax.clear()
        plot_data = data[2]
        plot_labels = data[1]
        if hasattr(plot_data, 'is_cuda') and plot_data.is_cuda:
            plot_data = plot_data.cpu()
        if hasattr(plot_data, 'numpy'):
            plot_data = plot_data.numpy()

        ticks = range(len(plot_data)-1, -1, -1)

        ax.barh(ticks, plot_data, align='center')

        if self.grid[position_id]:
            ax.grid(True)

        # set xlim
        ax.set_xlim(0, 1)

        # set y labels
        ax.set_yticks(ticks)
        ax.set_yticklabels(plot_labels)
        
        # set title
        if title is not None:
            ax.set_title(title)

        self.fig.tight_layout()
        self.fig.canvas.draw()
        self.ax[position_id] = ax
    
    def savefig(self, filename):
        figure_directory = os.path.join('results', 'elephant-cat', 'activation-plots')
        os.makedirs(figure_directory, exist_ok=True)
        figure_path = os.path.join(figure_directory, filename)
        self.fig.savefig(figure_path, bbox_inches='tight')


#### Test PlotGrid

In [ ]:
# test plotting function
plot_grid = PlotGrid(figsize=(9,3))

# initialize figures
# random
data = torch.rand((100))
plot_grid.plot((1, 2, 1), data, title='Random', xlim=(0, len(data)), ylim=(0.0, 1.0))
# normal
data = torch.randn((100))
plot_grid.plot((1, 2, 2), data, title='Normal', xlim=(0, len(data)), ylim=(-3.0, 3.0), filled=False)


In [ ]:
# update plots
for i in range(10):
    # random
    data = torch.rand((100))
    plot_grid.plot((1, 2, 1), data)
    # normal
    data = torch.randn((100))
    plot_grid.plot((1, 2, 2), data)


In [4]:
def pathJoin(*args):
    return os.path.abspath(os.path.join(*args))


def pprint(*args):
    pp.pprint(*args)


def rgetattr(obj, attr, *args):
    def _getattr(obj, attr):
        return getattr(obj, attr, *args)
    return functools.reduce(_getattr, [obj] + attr.split('.'))


toPILImage = transforms.ToPILImage()

softmax = torch.nn.Softmax(dim=1)


def predict(model, dataset, idx, grid, position):
    datapoint = dataset[idx]
    image = datapoint[1].unsqueeze(0).to(device)
    
    # convert to probabilities
    with torch.no_grad():
        output = softmax(model(image))
    
    # top 1
    prediction_probability, predicted_class = output.topk(1, 1, True, True)
    prediction_probability = prediction_probability.squeeze()
    predicted_class = predicted_class.squeeze()
    predicted_description = dataset.descriptions[predicted_class]
    top_1 = (predicted_class, predicted_description, prediction_probability)

    # top 5
    prediction_probability_5, predicted_class_5 = output.topk(5, 1, True, True)
    prediction_probability_5 = prediction_probability_5.squeeze()
    predicted_class_5 = predicted_class_5.squeeze()
    predicted_description_5 = [dataset.descriptions[predicted_class] for predicted_class in predicted_class_5]
    top_5 = (predicted_class_5, predicted_description_5, prediction_probability_5)

    # all
    prediction_probability_all, predicted_class_all = output.topk(output.shape[1], 1, True, True)
    prediction_probability_all = prediction_probability_all.squeeze()
    predicted_class_all = predicted_class_all.squeeze()
    predicted_description_all = [dataset.descriptions[predicted_class] for predicted_class in predicted_class_all]
    top_all = (predicted_class_all, predicted_description_all, prediction_probability_all)

    if position is not None:
        if grid is None:
            plt.subplot(*position)
            plt.imshow(toImage(datapoint[1]))
            plt.title('True: {}\n Predicted: {}'.format(datapoint[3], predicted_description))
        else:
            grid.plot(position, toImage(datapoint[1]), title='True: {}'.format(datapoint[3]))
            prediction_plot_position = (position[0], position[1], position[2] + 1)
            grid.prediction_plot(prediction_plot_position, top_5, title='Predicted: {}'.format(predicted_description))
    return (top_1, top_5, top_all)


#### Load Dataset - ImageNet

In [5]:
class BaseDataset(Dataset):

    def __init__(self, directory, split='train', transforms=None):
        self.datapoints = defaultdict(list)
        self.split = split
        self.directory = pathJoin(directory, split)
        self.datapoints = self.loadDataset()
        self.transforms = transforms

    def __len__(self):
        return len(self.datapoints)

    def __getitem__(self, idx):
        datapoint = self.loadDatapoint(idx)
        return datapoint

    def loadDatapoint(self, idx):
        raise NotImplementedError('Function "loadDatapoint" is not implemented')

    def loadDataset(self, name):
        raise NotImplementedError('Function "loadDataset" is not implemented')


In [6]:
class ImageNetDataset(BaseDataset):

    def __init__(self, directory, split='train', transforms=None):
        super().__init__(directory, split, transforms)
        self.descriptions = self.loadDescriptions()
        self.classes = self.loadClasses()
        self.groundtruths = self.loadValidationGroundtruths() if split == 'val' else []

    def loadDatapoint(self, idx):
        filepath = self.datapoints[idx]
        image = Image.open(filepath).convert('RGB')
        if self.split == 'val':
            groundtruth = self.groundtruths[idx]
        elif self.split == 'train':
            groundtruth = self.classes.index(filepath.split('/').pop().split('_')[0])
        if self.transforms:
            image = self.transforms(image)
        return (filepath, image, groundtruth, self.descriptions[groundtruth])

    def loadDataset(self):
        datapoints = []

        dataset_file_list_filename = 'ilsvrc2012{}.txt'.format(self.split)
        dataset_file_list_path = os.path.join(self.directory, dataset_file_list_filename)

        with open(dataset_file_list_path, 'r') as dataset_file_list_file:
            for line in tqdm(dataset_file_list_file, total=sum(1 for line in open(dataset_file_list_path))):
                file_path = pathJoin(self.directory, self.sanitizeFilename(line))
                datapoints.append(file_path)
        
        return datapoints
    
    def sanitizeFilename(self, filename):
        return filename.replace('"', '').strip()

    def loadDescriptions(self):
        descriptions = []

        descriptions_filename = 'synsets_with_descriptions.txt'
        descriptions_path = pathJoin(self.directory, '..', descriptions_filename)

        with open(descriptions_path, 'r') as descriptions_file:
            for line in descriptions_file:
                description_breakdown = line.split(' ')
                description_breakdown.pop(0)
                description = ' '.join(description_breakdown).strip()
                descriptions.append(description)

        return descriptions

    def loadValidationGroundtruths(self):
        groundtruths = []

        groundtruths_filename = 'validation_ground_truth.txt'
        groundtruths_path = pathJoin(self.directory, '..', groundtruths_filename)

        with open(groundtruths_path, 'r') as groundtruths_file:
            for line in groundtruths_file:
                groundtruth_breakdown = line.split(' ')
                groundtruth_breakdown.pop(0)
                groundtruth = ' '.join(groundtruth_breakdown).strip()
                groundtruths.append(int(groundtruth))

        return groundtruths

    def loadClasses(self):
        classes = []

        classes_filename = 'synsets.txt'
        classes_path = pathJoin(self.directory, '..', classes_filename)

        with open(classes_path, 'r') as classes_file:
            for line in classes_file:
                classes.append(line.strip())

        return classes


### Function to visualize images

In [7]:
class DeNormalize(object):
    # Source: https://discuss.pytorch.org/t/simple-way-to-inverse-transform-normalization/4821/3
    def __init__(self, mean, std):
        self.mean = mean
        self.std = std

    def __call__(self, image):
        """
        Args:
            tensor (Tensor): Tensor image of size (C, H, W) to be normalized.
        Returns:
            Tensor: Normalized image.
        """
        tensor = image.clone()
        for t, m, s in zip(tensor, self.mean, self.std):
            t.mul_(s).add_(m)
            # The normalize code -> t.sub_(m).div_(s)
        return tensor


In [8]:
BATCH_SIZE = 32

imagenet_normalization_values = {
    'mean': [0.485, 0.456, 0.406],
    'std': [0.229, 0.224, 0.225]
}

normalize = transforms.Normalize(**imagenet_normalization_values)
denormalize = DeNormalize(**imagenet_normalization_values)


def toImage(tensor_image):
    return toPILImage(denormalize(tensor_image))

test_transforms = transforms.Compose([
    transforms.Resize((224, 224)),
#     transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    normalize
])

imagenet_dataset_path = os.path.join('datasets', 'imagenet')

# imagenet_train_dataset = ImageNetDataset(imagenet_dataset_path, transforms=test_transforms)
imagenet_val_dataset = ImageNetDataset(imagenet_dataset_path, split='val', transforms=test_transforms)
# imagenet_test_dataset = ImageNetDataset(imagenet_dataset_path, split='test')

# imagenet_train_loader = DataLoader(imagenet_train_dataset, batch_size=BATCH_SIZE, shuffle=True)
imagenet_val_loader = DataLoader(imagenet_val_dataset, batch_size=BATCH_SIZE, shuffle=False)
# imagenet_test_loader = DataLoader(imagenet_test_dataset, batch_size=BATCH_SIZE, shuffle=False)


100%|██████████| 50000/50000 [00:01<00:00, 28053.39it/s]


#### Load Model

In [9]:
resnet50 = models.resnet50(pretrained=True)
resnet50.eval();

## Check Performance

In [10]:
def stats(prediction, target):
    total = prediction.size(0)
    prediction = prediction.t()
    correct = prediction.eq(target.view(1, -1).expand_as(prediction))
    top1 = correct[:1].view(-1).float().sum(0).item()
    top5 = correct[:5].view(-1).float().sum(0).item()
    return top1, top5, total

def score_model(model, dataloader):
    total_top1 = 0
    total_top5 = 0
    total_ = 0
    for batch in tqdm(dataloader):
        target = batch[2].to(device)
        input = batch[1].to(device)
        output = model(input)
        _, predicted_classes = output.topk(5, 1, True, True)
        top1, top5, total = stats(predicted_classes, target)
        total_top1 += top1
        total_top5 += top5
        total_ += total
    return total_top1/total_, total_top5/total_


In [11]:
score_model(resnet50, imagenet_val_loader)

100%|██████████| 1563/1563 [15:19<00:00,  1.02it/s]


(0.74548, 0.9201)

In [12]:
def load_model(model_name):

    model_urls = {
            'resnet50_trained_on_SIN': 'https://bitbucket.org/robert_geirhos/texture-vs-shape-pretrained-models/raw/6f41d2e86fc60566f78de64ecff35cc61eb6436f/resnet50_train_60_epochs-c8e5653e.pth.tar',
            'resnet50_trained_on_SIN_and_IN': 'https://bitbucket.org/robert_geirhos/texture-vs-shape-pretrained-models/raw/60b770e128fffcbd8562a3ab3546c1a735432d03/resnet50_train_45_epochs_combined_IN_SF-2a0d100e.pth.tar',
            'resnet50_trained_on_SIN_and_IN_then_finetuned_on_IN': 'https://bitbucket.org/robert_geirhos/texture-vs-shape-pretrained-models/raw/60b770e128fffcbd8562a3ab3546c1a735432d03/resnet50_finetune_60_epochs_lr_decay_after_30_start_resnet50_train_45_epochs_combined_IN_SF-ca06340c.pth.tar',
    }

    model = torchvision.models.resnet50(pretrained=False)
    model = torch.nn.DataParallel(model).cuda()
    checkpoint = model_zoo.load_url(model_urls[model_name])
    model.load_state_dict(checkpoint["state_dict"])
    return model

for model in ['resnet50_trained_on_SIN', 'resnet50_trained_on_SIN_and_IN', 'resnet50_trained_on_SIN_and_IN_then_finetuned_on_IN']:
    print('Model: {}'.format(model))
    model = load_model(model)
    model.eval()
    print(score_model(model, imagenet_val_loader))
    print('\n')


Model: resnet50_trained_on_SIN


100%|██████████| 1563/1563 [15:18<00:00,  1.94it/s]


(0.55478, 0.78786)


Model: resnet50_trained_on_SIN_and_IN


100%|██████████| 1563/1563 [15:17<00:00,  1.93it/s]


(0.72628, 0.90934)


Model: resnet50_trained_on_SIN_and_IN_then_finetuned_on_IN


100%|██████████| 1563/1563 [15:18<00:00,  1.90it/s]

(0.7543, 0.92456)




# Elephant Cat Dataset

In [ ]:
cat_filepath = os.path.join('elephant-cat-experiment', 'cat.png')
cat_image = Image.open(cat_filepath).convert('RGB')

stylized_cat_filepath = os.path.join('elephant-cat-experiment', 'stylized_cat.jpg')
stylized_cat_image = Image.open(stylized_cat_filepath).convert('RGB')

blog_cat_filepath = os.path.join('elephant-cat-experiment', 'cat7-elephant1.png')
blog_cat_image = Image.open(blog_cat_filepath).convert('RGB')

elephant_texture_filepath = os.path.join('elephant-cat-experiment', 'elephant_texture.jpg')
elephant_texture_image = Image.open(elephant_texture_filepath).convert('RGB')

In [ ]:
# asian_elephant, african_bush_elephant = None, None

# for index, i in enumerate(imagenet_val_dataset):
#     if asian_elephant is not None and african_bush_elephant is not None:
#         break
#     if i[3] == 'Asian elephant':
#         print('x {}'.format(index))
#         asian_elephant = i
#     if i[3] == 'African bush elephant':
#         print('y {}'.format(index))
#         african_bush_elephant = i

asian_elephant = imagenet_val_dataset[11246]
african_bush_elephant = imagenet_val_dataset[10618]

In [ ]:
plt.figure()
def show_image(image, position, title):
    plt.subplot(*position)
    plt.imshow(toImage(test_transforms(image)))
    plt.title(title)
    plt.axis('off')

show_image(cat_image, (2, 3, 1), 'original')
show_image(elephant_texture_image, (2, 3, 2), 'texture')
show_image(stylized_cat_image, (2, 3, 3), 'stylized')
show_image(blog_cat_image, (2, 3, 4), 'blog')
show_image(toImage(asian_elephant[1]), (2, 3, 5), 'asian elephant')
show_image(toImage(african_bush_elephant[1]), (2, 3, 6), 'african bush elephant')

In [ ]:
class Elephant_Cat_Dataset(list):
    def __new__(self, *args, **kwargs):
        return super(Elephant_Cat_Dataset, self).__new__(self, args, kwargs)

# override true class ids
asian_elephant = (asian_elephant[0], asian_elephant[1], 4, asian_elephant[3])
african_bush_elephant = (african_bush_elephant[0], african_bush_elephant[1], 5, african_bush_elephant[3])

elephant_cat_datapoints = [
    (None, test_transforms(cat_image), 0, 'original cat'),
    (None, test_transforms(elephant_texture_image), 1, 'elephant texture'),
    (None, test_transforms(stylized_cat_image), 2, 'stylized cat'),
    (None, test_transforms(blog_cat_image), 3, 'blog cat'),
    asian_elephant,
    african_bush_elephant
]

def create_elephant_cat_dataset():
    elephant_cat_dataset = Elephant_Cat_Dataset(elephant_cat_datapoints)

    setattr(elephant_cat_dataset, 'descriptions', imagenet_val_dataset.descriptions)
    return elephant_cat_dataset

elephant_cat_dataset = create_elephant_cat_dataset()

def test_model(model):
    elephant_cat_grid = PlotGrid(figsize=(9,18))
    for i in range(len(elephant_cat_datapoints)):
        predict(model, elephant_cat_dataset, i, elephant_cat_grid, (len(elephant_cat_datapoints), 2, ((i * 2) + 1) ));


In [ ]:
test_model(resnet50)

In [ ]:
def load_model(model_name):

    model_urls = {
            'resnet50_trained_on_SIN': 'https://bitbucket.org/robert_geirhos/texture-vs-shape-pretrained-models/raw/6f41d2e86fc60566f78de64ecff35cc61eb6436f/resnet50_train_60_epochs-c8e5653e.pth.tar',
            'resnet50_trained_on_SIN_and_IN': 'https://bitbucket.org/robert_geirhos/texture-vs-shape-pretrained-models/raw/60b770e128fffcbd8562a3ab3546c1a735432d03/resnet50_train_45_epochs_combined_IN_SF-2a0d100e.pth.tar',
            'resnet50_trained_on_SIN_and_IN_then_finetuned_on_IN': 'https://bitbucket.org/robert_geirhos/texture-vs-shape-pretrained-models/raw/60b770e128fffcbd8562a3ab3546c1a735432d03/resnet50_finetune_60_epochs_lr_decay_after_30_start_resnet50_train_45_epochs_combined_IN_SF-ca06340c.pth.tar',
    }

    model = torchvision.models.resnet50(pretrained=False)
    model = torch.nn.DataParallel(model)#.cuda()
    checkpoint = model_zoo.load_url(model_urls[model_name], map_location='cpu')
    model.load_state_dict(checkpoint["state_dict"])
    return model

for model in ['resnet50_trained_on_SIN', 'resnet50_trained_on_SIN_and_IN', 'resnet50_trained_on_SIN_and_IN_then_finetuned_on_IN']:
    print('Model: {}'.format(model))
    model = load_model(model)
    model.eval()
    test_model(model)
    print('\n')


#### Define Hook and Hooker Function

In [ ]:
def assign_plot_index(name, index, normalize, plot=False):
    def print_output(self, input, output):
        data = output.data.squeeze()
        if self.__class__.__name__ != 'Linear':
            data = data.norm(dim=(1, 2))
        if normalize:
            data = data.unsqueeze(0)
            data = data.renorm(1, 0, 1)
            data = data.squeeze()
        data = data.cpu().numpy()
        
        # value
        activations[name] = data
        
        # plot
        if plot:
            plt.subplot(*index)
            plt.plot(data)
            plt.fill_between(range(len(data)), data)
            plt.grid(True)
            plt.title(name)
    return print_output


#### add hooks to model

In [ ]:
def add_hooks(model, hooker, nrows=2, ncols=2, normalize=False):
    names = {}
    for name, param in model.named_parameters():
        name = name.split('.')
        name.pop()
        name = '.'.join(name)
        names[name] = None

    names = list(names)
    for index, name in enumerate(names):
        module = rgetattr(model, name)
        module.register_forward_hook(hooker(name, (nrows, ncols, index + 1), normalize))

# add hooks to resnet50
add_hooks(resnet50, assign_plot_index, nrows=54)

### Create/Load Activations

In [ ]:
values_directory = os.path.join('results', 'elephant-cat', 'activation-values')
os.makedirs(values_directory, exist_ok=True)

# for datapoint_index, datapoint in enumerate(elephant_cat_dataset):
#     activations = {}
#     top_1, top_5, top_all = predict(resnet50, elephant_cat_dataset, datapoint_index, None, None)
#     predicted_class, prediction_description, prediction_probability = top_1
#     true_class = datapoint[2]
#     values_path = os.path.join(values_directory, '{}-{}-{}.pickle'.format(datapoint_index, true_class, predicted_class))
#     pickle.dump(activations, open(values_path, 'wb'))

def get_sort_key(name):
    return int(name.split('/')[-1].split('.')[0].split('-')[0])

activations_file_list = sorted(
    [os.path.join(values_directory, file) for file in listdir(values_directory)],
    key=get_sort_key
)

In [ ]:
def plot_activations(activations, datapoint_index, true_class, predicted_class):
    activation_grid = PlotGrid(figsize=(9,90))
    for index, name in enumerate(activations):
        data = activations[name]
        activation_grid.plot((54, 2, index+1), data, title=name)

    # plot image
    activation_grid.plot(
        (54, 2, 108),
        toImage(elephant_cat_dataset[datapoint_index][1]),
        title='True: {}\n Predicted: {}'.format(
            elephant_cat_dataset[datapoint_index][3],
            elephant_cat_dataset.descriptions[predicted_class]
        )
    )

    # save figure
    activation_grid.savefig('{}-{}-{}.png'.format(datapoint_index, true_class, predicted_class))


def load_activations(activation_file_index):
    activations_file_path = activations_file_list[activation_file_index]
    imagenet_val_index, true_class, predicted_class = activations_file_list[activation_file_index].split('/')[-1].split('.')[0].split('-')
    return pickle.load(open(activations_file_path, 'rb')), int(imagenet_val_index), int(true_class), int(predicted_class)



In [ ]:
for index, name in enumerate(activations_file_list):
    activations, datapoint_index, true_class, predicted_class = load_activations(index)
    plot_activations(activations, datapoint_index, true_class, predicted_class)


## Jensen-Shannon Divergence

In [ ]:
def JSDiv(p, q):
    return torch.nn.KLDivLoss(reduction='sum')(p.log(), q) + torch.nn.KLDivLoss(reduction='sum')(q.log(), p)

# test inputs
p = torch.FloatTensor([0.3, 0.6, 0.1])
q = torch.FloatTensor([0.4, 0.5, 0.1])

# test call
print('{0:.4f} JS Divergence'.format(JSDiv(p, q).item()))

#### Test on activations

In [ ]:
def get_layer_activation(activations, layer_name):
    data = torch.from_numpy(activations[layer_name])
    if layer_name == 'fc':
        data = torch.nn.Softmax(dim=0)(data)
    else:
        data = data.unsqueeze(0)
        data = data.renorm(1, 0, 1)
        data = data.squeeze()
    return data


In [ ]:
activations_1, _, _, _ = load_activations(1)
activations_2, _, _, _ = load_activations(2)

layer_names = list(activations_2.keys())
# regex = re.compile(r'conv1|fc')
# layer_names = list(filter(regex.search, layer_names))
# layer_indices = [0, 1, 4, 8, 14, 17]
# layer_names = [layer_names[i] for i in layer_indices]
layer_index = -2

p = get_layer_activation(activations_1, layer_names[layer_index])
q = get_layer_activation(activations_2, layer_names[layer_index])

assert np.isclose(p.sum().item(), 1), 'activations (p) is not a distribution'
assert np.isclose(q.sum().item(), 1), 'activations (q) is not a distribution'

print('{0:.4f} JS Divergence of activations at layer "{1}"'.format(JSDiv(q, p).item(), layer_names[layer_index]))

#### Calculate divergence for multiple images given a reference image

In [ ]:
def activation_divergence(target_index, compare_indices, layer_name):
    activations, imagenet_val_index, true_class, predicted_class = load_activations(target_index)
    p = get_layer_activation(activations, layer_name)
    divergence_information = []
    for compare_index in compare_indices:
        activations_, imagenet_val_index_, true_class_, predicted_class_  = load_activations(compare_index)
        q = get_layer_activation(activations_, layer_name)
        divergence_information.append((
            JSDiv(p, q).item(),
            true_class==true_class_,
            predicted_class==predicted_class_,
            true_class==predicted_class,
            imagenet_val_index,
            imagenet_val_index_
        ))
    return divergence_information


In [ ]:
divergence_values_directory = os.path.join('results', 'elephant-cat', 'divergence-values')
os.makedirs(divergence_values_directory, exist_ok=True)

divergence_figure_directory = os.path.join('results', 'elephant-cat', 'divergence-plots')
os.makedirs(divergence_figure_directory, exist_ok=True)

def get_sort_key(name):
    return int(name.split('/')[-1].split('.')[0].split('-')[1])

divergences_file_list = sorted(
    [os.path.join(divergence_values_directory, file) for file in listdir(divergence_values_directory)],
    key=get_sort_key
)

#### Calculate/Plot activation divergence at specified layers given the reference image over the validation set

In [ ]:
def calculate_divergences(datapoint_index, save=True):
    other_indices = list(range(0, len(elephant_cat_dataset)))
    other_indices.remove(datapoint_index)
    divergence_information = {}
    for index, layer_name in enumerate(layer_names, 1):
        divergence_information[layer_name] = activation_divergence(datapoint_index, other_indices, layer_name)
    if save:
        values_path = os.path.join(divergence_values_directory, 'divergence-{}.pickle'.format(datapoint_index))
        pickle.dump(divergence_information, open(values_path, 'wb'))
    return divergence_information


def load_divergences(divergences_file_index):
    divergences_file_path = divergences_file_list[divergences_file_index]
    _, imagenet_val_index = divergences_file_list[divergences_file_index].split('/')[-1].split('.')[0].split('-')
    return pickle.load(open(divergences_file_path, 'rb')), int(imagenet_val_index)


def plot_divergences(divergence_information, datapoint_index, cols=2, same_class_size=100, other_class_size=10, save=True):
    for index, layer_name in enumerate(layer_names, 1):
        divergences = [ x[0] for x in divergence_information[layer_name] ]
        class_matches = [ int(x[1]) for x in divergence_information[layer_name] ]
        class_matches_size = [ same_class_size if x==1 else other_class_size for x in class_matches ]
        plt.subplot(len(layer_names)//cols + len(layer_names)%cols, cols, index)
        plt.scatter(range(len(divergences)), divergences, s=class_matches_size, c=class_matches)
        other_indices = list(range(0, len(elephant_cat_dataset)))
        other_indices.remove(datapoint_index)
        plt.xticks(np.arange(len(elephant_cat_datapoints) - 1), [elephant_cat_datapoints[other_index][3] for other_index in other_indices], rotation=45)
        plt.title(layer_name)
        plt.tight_layout()
        plt.show()
    if save:
        plt.savefig(os.path.join(divergence_figure_directory, 'divergence-{}.png'.format(datapoint_index)), bbox_inches='tight')


#### Calculate Min/Mean/Max Divergence

In [ ]:
def remove_zeros(array):
    return array[np.flatnonzero(array)]

def calculate_divergence_metrics(divergence_information):
    metrics = {}
    for index, layer_name in enumerate(layer_names, 1):
        divergences = np.array([ x[0] for x in divergence_information[layer_name] ])
        class_matches = np.array([ int(x[1]) for x in divergence_information[layer_name] ])
        other_class = (class_matches + 1) % 2
        class_divergences = remove_zeros(divergences * class_matches)
        other_divergences = remove_zeros(divergences * other_class)
        metrics[layer_name] = {
            'same': {
                'min': class_divergences.min(),
                'mean': class_divergences.mean(),
                'max': class_divergences.max()
            },
            'other': {
                'min': other_divergences.min(),
                'mean': other_divergences.mean(),
                'max': other_divergences.max()
            }
        }
    return metrics


#### Calculate activation divergences

In [ ]:
divergence_metrics = {}

for datapoint_index in range(len(elephant_cat_dataset)):
    divergence_information = calculate_divergences(datapoint_index)
#     divergence_metrics[datapoint_index] = calculate_divergence_metrics(divergence_information)
#     pp.pprint(divergence_information)

metrics_path = os.path.join(divergence_values_directory, 'divergence-metrics.pickle')
pickle.dump(divergence_metrics, open(metrics_path, 'wb'))

#### Plot activation divergences

In [ ]:
for index, name in enumerate(divergences_file_list):
    plt.figure(figsize=(9, len(layer_names) * 2))
    divergence_information, datapoint_index = load_divergences(index)
    plot_divergences(divergence_information, datapoint_index)
